In [1]:
import torch
import yaml
import os
from math import sqrt
from sklearn.metrics import mean_squared_error
from scipy.stats.stats import pearsonr
from utils.data_utils import *
import numpy as np

In [2]:
"""
选择数据集
"""
dataset = 'twitter'
client_num = 3 #机构数量
local_log_path = os.path.join('.', 'local_log', 'GAT_twitter_w20h7c3')


In [3]:
"""
centralized模型及其各个子机构的结果
"""
institution_name = dataset + '-global'
_, institution_list = gen_institution(dataset,client_num)
y_true_states = []
y_pred_states = []
with open(os.path.join(local_log_path, institution_name+'_runTag' + str(-1)+"_log.yaml"), "r") as f:
    rel_log=yaml.safe_load(f)
    y_true_states = np.array(rel_log['y_true_states'])
    y_pred_states = np.array(rel_log['y_pred_states'])
    y_true = np.reshape(y_true_states, (-1))
    y_pred = np.reshape(y_pred_states, (-1))
    metrics = regression_metrics(y_true,y_pred)
    print("Centralized model: ",metrics)

for i in range(0,client_num):
    institueion_idx = institution_list[i]
    y_true = np.reshape(y_true_states[:, institueion_idx], (-1))
    y_pred = np.reshape(y_pred_states[:, institueion_idx], (-1))
    metrics = regression_metrics(y_true,y_pred)
    print("Centralized model ({}):".format(chr(ord('A') + i)), metrics)

Centralized model:  {'mse': 1830365.1298534586, 'rmse': 1352.909874992957, 'mae': 486.5475042891378, 'mape': 0.8920565379025248, 'pcc': 0.9568081617525952, 'r2': 0.9135341921702629}
Centralized model (A): {'mse': 483141.3306413577, 'rmse': 695.0836860705031, 'mae': 416.27156710875664, 'mape': 0.5832232609367072, 'pcc': 0.877596086202056, 'r2': 0.7607661844578033}
Centralized model (B): {'mse': 3231750.4543137522, 'rmse': 1797.7069990167342, 'mae': 501.69616611053544, 'mape': 1.266713918353391, 'pcc': 0.9585678415689818, 'r2': 0.9162404679625922}
Centralized model (C): {'mse': 1864212.070284789, 'rmse': 1365.3615163336005, 'mae': 583.1444946202365, 'mape': 0.812420121014793, 'pcc': 0.9625470950413267, 'r2': 0.9240742057934431}


In [5]:
"""
将子机构的结果整合，求在所有节点上的平均效果
"""
runTag = 0 #可以是-1，0，1...
y_true = []
y_pred = []
for institueion_idx in range(0,client_num):
    institution_name = dataset + '-sub' + chr(ord('A') + institueion_idx)
    with open(os.path.join(local_log_path, institution_name+'_runTag' + str(runTag)+"_log.yaml"), "r") as f:
        rel_log=yaml.safe_load(f)
        y_true_states = np.reshape(np.array(rel_log['y_true_states']), (-1))
        y_pred_states = np.reshape(np.array(rel_log['y_pred_states']), (-1))
        y_true = np.append(y_true, y_true_states)
        y_pred = np.append(y_pred, y_pred_states)
        metrics = regression_metrics(y_true_states,y_pred_states)
        print("{}:".format(institution_name), metrics)

metrics = regression_metrics(y_true,y_pred)
print("global view for local models: ", metrics)

twitter-subA: {'mse': 522009.9446345047, 'rmse': 722.5025568359636, 'mae': 412.0730777113061, 'mape': 0.5609449503604691, 'pcc': 0.8640256504820671, 'r2': 0.7415198765129342}
twitter-subB: {'mse': 3900144.039408493, 'rmse': 1974.878234071279, 'mae': 537.0357583668497, 'mape': 1.3411400350980947, 'pcc': 0.9571845520840634, 'r2': 0.8989172449304386}
twitter-subC: {'mse': 2307104.4825579985, 'rmse': 1518.91556136541, 'mae': 650.2360830740495, 'mape': 0.7699654932177586, 'pcc': 0.954175587473578, 'r2': 0.9060360444244073}
global view for local models:  {'mse': 2197894.395115551, 'rmse': 1482.5297282400616, 'mae': 513.5131050194303, 'mape': 0.9014186482085409, 'pcc': 0.9486145479007393, 'r2': 0.8961722383700936}


In [6]:
"""
比较integrated模型相较于local模型提升了多少
"""
runTag = 0 #integrated模型的runTage>-1
criteria = 'pcc' #模型平均标准


rel_local, rel_integrated = [], []
rel_local_ins, rel_integrated_ins = [], []
for institueion_idx in range(0,client_num):
    institution_name = dataset + '-sub' + chr(ord('A') + institueion_idx)
    with open(os.path.join(local_log_path, institution_name+'_runTag' + str(runTag)+"_log.yaml"), "r") as f:
        rel_log=yaml.safe_load(f)
        y_true_states = np.array(rel_log['y_true_states'])
        y_pred_states = np.array(rel_log['y_pred_states'])
        # 机构级别结果
        metrics = regression_metrics(np.reshape(y_true_states, (-1)), np.reshape(y_pred_states, (-1)))
        rel_integrated_ins.append(metrics[criteria])
        # 节点级别结果
        for i in range(y_pred_states.shape[1]):
            y_true = np.reshape(y_true_states[:,i], (-1))
            y_pred = np.reshape(y_pred_states[:,i], (-1))
            metrics = regression_metrics(y_true,y_pred)
            rel_integrated.append(metrics[criteria])

    with open(os.path.join(local_log_path, institution_name+'_runTag' + str(-1)+"_log.yaml"), "r") as f:
        rel_log=yaml.safe_load(f)
        y_true_states = np.array(rel_log['y_true_states'])
        y_pred_states = np.array(rel_log['y_pred_states'])
        # 机构级别结果
        metrics = regression_metrics(np.reshape(y_true_states, (-1)), np.reshape(y_pred_states, (-1)))
        rel_local_ins.append(metrics[criteria])
        # 节点级别结果
        for i in range(y_pred_states.shape[1]):
            y_true = np.reshape(y_true_states[:,i], (-1))
            y_pred = np.reshape(y_pred_states[:,i], (-1))
            metrics = regression_metrics(y_true,y_pred)
            rel_local.append(metrics[criteria])

print("Local:", rel_local_ins)
print("Integrated:", rel_integrated_ins)

rel_local = np.array(rel_local)
rel_integrated = np.array(rel_integrated)
rel_local_ins = np.array(rel_local_ins)
rel_integrated_ins = np.array(rel_integrated_ins)

print("{}下降节点数量:{}".format(criteria,sum(rel_integrated<rel_local)))
print("{}不变节点数量:{}".format(criteria,sum(rel_integrated==rel_local)))
print("{}上升节点数量:{}".format(criteria,sum(rel_integrated>rel_local)))
print("节点总数:{}".format(len(rel_local)))

print("{}下降机构数量:{}".format(criteria,sum(rel_integrated_ins<rel_local_ins)))
print("{}不变机构数量:{}".format(criteria,sum(rel_integrated_ins==rel_local_ins)))
print("{}上升机构数量:{}".format(criteria,sum(rel_integrated_ins>rel_local_ins)))
print("机构总数:{}".format(len(rel_local_ins)))

Local: [0.8698470035013282, 0.9571845520840634, 0.9205679495045057]
Integrated: [0.8640256504820671, 0.9571845520840634, 0.954175587473578]
pcc下降节点数量:14
pcc不变节点数量:18
pcc上升节点数量:16
节点总数:48
pcc下降机构数量:1
pcc不变机构数量:1
pcc上升机构数量:1
机构总数:3
